# LakeCREST notebooks
*Version: 08.04.2022 14:45*
## 2. Explore subset (LSWT)
In this script we will use [**xarray**](https://docs.xarray.dev/en/latest/index.html) and [**dask**](https://dask.org/) to explore LSWT lake subset of the [**ESA CCI Lakes v1.1**](https://catalogue.ceda.ac.uk/uuid/ef1627f523764eae8bbb6b81bf1f7a0a).

### 2.1 Importing modules
First, we import the necessary python modules for the following steps.

In [ ]:
import pathlib
import xarray as xr
import numpy as np
import time
import pandas as pd
from dask.distributed import Client

### 2.2 Load Dataset
Show a list of all available lake subsets in the subset folder.

In [ ]:
# Search for available .zarr folders
path_ssdir = path_wrk = pathlib.Path().absolute().joinpath('subsets')
ss_list = sorted(path_ssdir.glob('*.zip'))

print('The following lake subsets are currently available: ')
for ss in ss_list: print(f'- {ss.stem.split("-")[-2]: <13}\t({ss.stem.split("-")[-1]})')

#### Define study lake (✦ User inputs)
Select the desired study lake and ESA CCI Lakes version number.

In [ ]:
# User input: Define lake
lakename = 'Michigan'
version ='1.1' # ='2.0'

Now we can look for the subset in the \subsets folder and load it.

In [ ]:
# Define subset filepath
path_wrk = pathlib.Path().absolute()
path_ss = path_wrk.joinpath(fr'subsets\ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-LSWT_LIC-{lakename}-fv{version}.zip')

if not(path_ss.exists()):
    # If file doesn't exist throw error
    print(f'Error: No subset for Lake {lakename} found in subset folder!')
else:
    # Load dataset
    print(f'Subset for Lake {lakename} found in subset folder and loaded.')
    DS = xr.open_zarr(store=path_ss)
#     DS = xr.open_dataset(filename_or_obj=path_ss,
#                          engine='netcdf4',
#                          decode_cf=True,
#                          chunks='auto'
#                         )

Preview the loaded lake data.

In [ ]:
DS

### 2.3 Preprocessing data
#### LSWT: Mask by quality flag
Using the variable 'lswt_quality_level' we can mask out cells that don't fulfill the necessary quality criteria for lake surface water quality that ranges from 0 to 5:

- 0: unprocessed 
- 1: bad 
- 2: marginal 
- 3: intermediate
- 4: good 
- 5: best

Within the [D4.1: Product Validation and Intercomparison Report](https://climate.esa.int/media/documents/CCI-LAKES-0031-PVIR_v1.4.pdf) the authors of the ESA CCI Lakes product recommend to limit the use of LSWT for lake-climate applications to quality flags 4-5 (good and best).

#### LIC: Mask by class
We also mask out all non-ice pixels in the LIC data using the given classification scheme ranging from 1 to 4:

- 1: water 
- 2: ice 
- 3: cloud
- 4: bad 

Then we can add reasign nan cells that are ice-cells to LSWT with the temperature 0°C/273.15°K.

In [ ]:
qmask = (DS['lswt_quality_level']>1)
lswt_masked = DS['lake_surface_water_temperature'].where(cond=qmask)

# Set values where there's ice to 0°C/273.15°K for the LSWT variable
lswt_masked = xr.where(DS['lake_ice_cover']==2, 273.15, lswt_masked, keep_attrs=False)
lswt_masked = lswt_masked.rename('lswt')

#### Drop days with no data
We can drop days with all-nan values for the variable 'lake_surface_water_temperature' by using the [**xarray.Dataset.dropna**](https://docs.xarray.dev/en/latest/generated/xarray.Dataset.dropna.html) function with the parameter ***how='all'***.

In [ ]:
lswt_masked_dropna = lswt_masked.dropna(dim='time', 
                                        how='all')

dropcount = lswt_masked.time.size - lswt_masked_dropna.time.size

print(f'Dropped {dropcount} ({dropcount/lswt_masked.time.size*100:0.1f}%) days ' \
      'with all-masked cells (from {lswt_masked.time.size} to {lswt_masked_dropna.time.size})')

lswt_masked_dropna

### 2.4 Plotting
#### 2.4.1 LSWT Time-series animation
Let's explore the time-series LSWT data. To do this we first convert the [xarray.Dataset](https://docs.xarray.dev/en/latest/generated/xarray.Dataset.html) to a [xarray.Dataarray](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.html) containing only the variable 'lake_surface_water_temperature' and convert the value from °K to °C.

In [ ]:
lswt = lswt_masked_dropna-273.15

Now we can create an interactive animation that displays the temperature data in a cartographic reference system using [Holoviews](https://holoviews.org/) and [Cartopy](https://scitools.org.uk/cartopy/docs/latest/).

In [ ]:
import hvplot.xarray
import holoviews as hv
import cartopy.crs as ccrs

# Define CRS
crs = ccrs.PlateCarree(central_longitude=0, globe=None)

# Plot time-series data
plot_lswt = lswt.hvplot(
    geo=True, tiles='CartoLight',
    groupby="time",  # adds a widget for time
    clim=(0, 25),  # sets colormap limits
    crs=crs,
    cmap='jet',
    widget_type="scrubber",
    widget_location="bottom",
    #width=300,
    xlabel='Longitude', ylabel='Latitude',
    #title=f'ESA CCI Lakes v1.1\nLake {lakename} LSWT animation',
    clabel='Lake surface water temperature (°C)'
)

plot_lswt

#### LSWT Time-series animation (weekly aggregates)
For many days the data availability is strongly impaired by cloud coverage. One possibility to deal with this is to trade the temporal resolution for data availability and create weekly aggregates out of the daily dataset. Note, weeks with all all-nan days are excluded.

In [ ]:
import hvplot.xarray
import holoviews as hv
import cartopy.crs as ccrs

# Create weekly aggegrates by resampling from 
# daily to weekly and applying the weekly mean over time
lswt_weekly = lswt.resample(time="W-MON").mean(dim=["time"], skipna=True)

# Define CRS
crs = ccrs.PlateCarree(central_longitude=0, globe=None)

# Plot time-series data
plot_lswt_weekly = lswt_weekly.hvplot(
    geo=True, tiles='CartoLight',
    groupby="time",  # adds a widget for time
    clim=(0, 25),  # sets colormap limits
    crs=crs,
    cmap='jet',
    widget_type="scrubber",
    widget_location="bottom",
    #width=300,
    xlabel='Longitude', ylabel='Latitude',
    #title=f'ESA CCI Lakes v1.1\nLake {lakename} LSWT animation',
    clabel='Lake surface water temperature (°C)'
)

plot_lswt_weekly

 #### 2.4.2 Setup plot export function based on Bokeh

In [ ]:
def exportPlot(plot, fn):
    """Takes plot and saves it as .png and .svg in subfolder with the provided filename"""
    import holoviews as hv
    import cairosvg
    from bokeh.io import export_svgs
    
    #set the backend and the renderer
    hv.extension('bokeh')
    br = hv.renderer('bokeh')

    #export the plot
    plot = br.get_plot(plot)
    plot = plot.state
    plot.output_backend='svg'

    # check if svgs dir is already created
    pathlib.Path('svgs').mkdir(parents=False, exist_ok=True)
    pathlib.Path('pngs').mkdir(parents=False, exist_ok=True)
    path_svg = rf'svgs\{fn}.svg'
    path_png = rf'pngs\{fn}.png'
    export_svgs(plot, filename=path_svg)
    cairosvg.svg2png(url=path_svg, write_to=path_png, scale=2.0)

    print(f'Exported .svg to "{pathlib.Path().absolute().joinpath(path_svg)}".')
    print(f'Exported .png to "{pathlib.Path().absolute().joinpath(path_png)}".')

#### 2.4.3 Mean LSWT
#### Mean LSWT plot
We can use [**xarray.DataArray.mean**](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.mean.html) with the parameters ***dim*** and ***skipna*** to plot the spatially aggregated daily LSWT mean over time.

In [ ]:
import hvplot.xarray

# Slice data with timeslice
timeslice = slice('2006-01-01', '2007-12-31')
lswt_slice = lswt.sel(time=timeslice)

# Compute daily means and load to memory
lswt_slice_mean = lswt_slice.mean(dim=["lat", "lon"], skipna=True).compute()

# Create a line plot using hvplot
lineplot = lswt_slice_mean.hvplot.line(ylabel='Mean LSWT (°C)', 
                                       title=f'ESA CCI Lakes v{version}\nLake {lakename} LSWT mean')

# Create a scatter plot using hvplot
scatterplot = lswt_slice_mean.hvplot.scatter(c='k')

# Overlay plots to create output
combined = lineplot * scatterplot
combined

In [ ]:
# Export plot
fn = f'{lakename}-LSWT-MeanScatter-{timeslice.start}-{timeslice.stop}'
exportPlot(combined, fn)

#### Mean LSWT plot (by coverage)
As we can see there are days with outliers. Especially, days with very low data coverage and missclassified ice cells. We can set a treshold for a minimal lake coverage and replot the data.

In [ ]:
import holoviews as hv
import warnings

# Slice data with timeslice
timeslice = slice('2006-01-01', '2007-12-31')
lswt_slice = lswt.sel(time=timeslice)

# Ignore All-NaN slice warnings because they are expected
warnings.filterwarnings(action='ignore', message='All-NaN slice encountered')

# Compute daily means and load to memory
lswt_slice_mean = lswt_slice.mean(dim=["lat", "lon"], skipna=True).compute()

# Setup treshold for minimal lake coverage
coverage_treshs = [20, 40, 60, 80] # treshold in %

# Calculate cell treshold to keep daily mean
mask = ~np.isnan(lswt_slice.max(dim='time', skipna=True))
lakecells = np.count_nonzero(mask) # count of lakecells

lineplots = []
scatterplots = []

for tresh in coverage_treshs:
    nancells_tresh = int(lakecells*((tresh)/100)) # minimal cells treshold
    # Use xarray.DataArray.dropna with threshold to get rid of days with low-coverage
    lswt_slice_tresh = lswt_slice.dropna(dim='time', thresh=nancells_tresh)
    # Compute daily means
    lswt_slice_tresh_mean = lswt_slice_tresh.mean(dim=["lat", "lon"], skipna=True)
    # Create line and scatter plot for treshold
    lineplots.append(lswt_slice_tresh_mean.hvplot.line(label=f'{tresh}%'))
    scatterplots.append(lswt_slice_tresh_mean.hvplot.scatter())
    print(f'Dropping days with less than {nancells_tresh} valid cells '\
      f'({tresh}% of Lake {lakename}): ', end='')
    print(f'Dropped {(100-(lswt_slice_tresh_mean.size/lswt_slice_mean.size)*100):0.1f}% of available daily datapoints.')

combined = lineplots+scatterplots
combined = hv.Overlay(combined)
combined.opts(width=900, ylabel='LSWT (°C)', title=f'ESA CCI Lakes v{version}\nLake {lakename} LSWT mean (by coverage)')

In [ ]:
# Export plot
fn = f'{lakename}-LSWT-MeanScatterByCov-{timeslice.start.replace("-", "")}-{timeslice.stop.replace("-", "")}'
exportPlot(combined, fn)

#### 2.4.4 Quality mask comparison

Setup two quality masked datasets with one masked from good-best and one from marginal-best.

In [ ]:
import hvplot.xarray

# Quality mask 1 from good-best (4-5)
qmask1 = (DS['lswt_quality_level']>3)
DS_qmask1 = DS.where(cond=qmask1)
DS_qmask1_dropna = DS_qmask1.dropna(dim='time', how='all', subset=['lake_surface_water_temperature'])
lswt1 = DS_qmask1_dropna['lake_surface_water_temperature']-273.15

# Quality mask 2 from marginal-best (2-5)
qmask2 = (DS['lswt_quality_level']>1)
DS_qmask2 = DS.where(cond=qmask2)
DS_qmask2_dropna = DS_qmask2.dropna(dim='time', how='all', subset=['lake_surface_water_temperature'])
lswt2 = DS_qmask2_dropna['lake_surface_water_temperature']-273.15

# Set colors
color1 = '#3366CC'
color2 = '#DC3912'

# Slice data with timeslice
timeslice = slice('2006-01-01', '2007-12-31')
lswt1_slice = lswt1.sel(time=timeslice)
lswt2_slice = lswt2.sel(time=timeslice)

# Compute daily means and load to memory
lswt1_slice_mean = lswt1_slice.mean(dim=["lat", "lon"], skipna=True).compute()
lswt2_slice_mean = lswt2_slice.mean(dim=["lat", "lon"], skipna=True).compute()

Get the relative data availability/lake coverage for the two datasets.

In [ ]:
# Create function to get daily rel. lake coverage
def getRelCoverage(da):
    """Counts valid cells of xr.DataArray and returns rel. coverage"""
    warnings.filterwarnings(action='ignore', message='All-NaN slice encountered') # all-nan expected
    # Calculate cell treshold to keep daily mean
    mask = ~np.isnan(da.max(dim='time', skipna=True))
    lakecells = np.count_nonzero(mask) # count of lakecells
    validCellsArray = (~np.isnan(da))
    validCellsRelCountArray = (validCellsArray.sum(dim=['lat', 'lon'])/lakecells)*100       
    return(validCellsRelCountArray)

coverage1 = getRelCoverage(DS_qmask1['lake_surface_water_temperature'].sel(time=timeslice)-273.15)
coverage2 = getRelCoverage(DS_qmask2['lake_surface_water_temperature'].sel(time=timeslice)-273.15)

Plot two datasets together in scatterplot.

In [ ]:
# Create lineplots
lineplot1 = lswt1_slice_mean.hvplot.line(c=color1)
lineplot2 = lswt2_slice_mean.hvplot.line(c=color2)

# Create scatterplots
scatterplot1 = lswt1_slice_mean.hvplot.scatter(c=color1, label='best-good', line_width=1, marker='triangle_dot', alpha=0.5)
scatterplot2 = lswt2_slice_mean.hvplot.scatter(c=color2, label='marginal-best', line_width=1, marker='triangle_dot', alpha=0.5)

# Create histograms
#hist1 = coverage1.hvplot.area(color=color1, alpha=0.2)

# Overlay plots to create output
combined = lineplot2*lineplot1*scatterplot2*scatterplot1 #*hist1

combined.opts(
    axiswise=True,
    width=900,
    ylabel='Mean LSWT (°C)',
    xlabel='Date',
    title=f'ESA CCI Lakes v{version}\nLake {lakename} LSWT mean (by quality, without LIC)')

combined

In [ ]:
# Export plot
fn = f'{lakename}-LSWT-MeanScatterByQual-{timeslice.start.replace("-", "")}-{timeslice.stop.replace("-", "")}'
exportPlot(combined, fn)

#### 2.4.7 Visualize data availability
As we can see in the mean plots the data availability changes in accordance with missions timeline of data sources. We can use the time dimension to calculate the yearly mean coverage frequency per cell.

In [ ]:
import dask.array as da
import warnings

# Setup function to compute mean time-delay of non-nan values along
def getCovFreq(darr):
    """Takes xarray.DataArray and returns dataarray with mean timedelay (d) between valid values"""
    lat_sz, lon_sz = darr.lat.size, darr.lon.size # get lat and lon sizes
    mask = darr.to_masked_array() # create a np.masked_array from xr.dataarray

    t = darr.time.values # get time dimension as array
    t_steps = len(darr.time)

    arr_t3d = np.broadcast_to(t[:, np.newaxis, np.newaxis], (t_steps, lat_sz, lon_sz)) # expand time dimension to match dataarray
    arr_t3d_masked = np.ma.masked_where(mask.mask, arr_t3d).filled(np.datetime64('NaT'))  # mask it and set masked cells to nan
    darr_t3d_masked = da.from_array(arr_t3d_masked, chunks='auto') # convert np.arr to dask-array

    def mean_cfreq(x):
        """Takes np.datetime64 array and returns np.float32 array with temp-diff (d), dropping nat-values"""
        arr = x[~np.isnat(x)] # drop nats
        diff_td64 = np.diff(arr)
        diff_d = (diff_td64 / np.timedelta64(1,'D')).astype(np.float32)
        warnings.filterwarnings(action='ignore', message='Mean of empty slice')
        return np.nanmean(diff_d) # return mean of differences

    # Apply mean_cfreq along time-dimension
    cfreq = np.apply_along_axis(mean_cfreq, 0, arr_t3d_masked)
    xr_cfreq = xr.DataArray(data=cfreq, coords=(darr.lat.values, darr.lon.values), dims=('lat', 'lon'), name='mean_cfreq')
    return(xr_cfreq)

# Slice data with timeslice
timeslice = slice('2005-01-01', '2008-12-31')
lswt_slice = lswt.sel(time=timeslice)

# Group LSWT by years and compute the yearly mean cell coverage frequency
lswt_cfrwq = lswt_slice.groupby("time.year").map(getCovFreq)

#### Yearly coverage frequency - Violin plot

In [ ]:
# Create a violin plot of the distribution of yearly mean coverage frequency
plot = lswt_cfrwq.hvplot.violin(y='mean_cfreq', by='year', 
                                title=f'ESA CCI Lakes v{version}\nLake {lakename} - Coverage frequency', 
                                ylabel='Yearly coverage frequency (d)', 
                                grid=True, 
                                ylim=[0,30],
                               )
plot

In [ ]:
# Export plot
fn = f'{lakename}-LSWT-CoverageFreq-{timeslice.start}-{timeslice.stop}'
exportPlot(plot, fn)

#### Yearly coverage frequency - Yearly mean maps

In [ ]:
# Plot yearly mean coverage frequency maps 
lswt_cfrwq.hvplot(geo=True, 
                  tiles='CartoLight',
                  groupby="year",  # adds a widget for time
                  clim=(0, 25),  # sets colormap limits
                  crs=crs,
                  cmap='jet',
                  widget_type="scrubber",
                  widget_location="bottom",
                  #height=400,
                  xlabel='Longitude', ylabel='Latitude',
                  #title=f'ESA CCI Lakes v1.1\nLake {lakename} LSWT coverage frequency',
                  clabel='mean coverage frequency (d)'
)

#### Yearly coverage frequency - Stacked bar plot
First we calculate the relative daily lake coverage by quality flag.

In [ ]:
import warnings

# Colorlist with nice colors
colorlist = ['#66AA00','#AAAA11', '#FF9900', '#E67300', '#DC3912']

# Create flag count function
def countFlags(ds):
    """Counts quality flags and returns xr.DataArray with rel. counts as variables"""
    warnings.filterwarnings(action='ignore', message='All-NaN slice encountered') # all-nan expected
    # Calculate cell treshold to keep daily mean
    mask = ~np.isnan(ds['lake_surface_water_temperature'].max(dim='time', skipna=True))
    lakecells = np.count_nonzero(mask) # count of lakecells
    
    qflags = list(range(1,6)) #list from 1-5
    counts = []
    labels = ['best', 'good', 'medium', 'marginal', 'bad']
    for flag in qflags:
        maskedArray = (ds['lswt_quality_level']==flag)
        countArray = (maskedArray.sum(dim=['lat', 'lon'])/lakecells)*100
        counts.append(countArray.rename(labels[flag-1]))
        
    return(xr.merge(counts))

# Run countFlags() on dataset
count = countFlags(DS).load()

Then we can plot the relative daily coverage per quality flag in a barplot.

In [ ]:
from bokeh.models import FuncTickFormatter

# Colorlist with nice colors
colorlist = ['#66AA00','#AAAA11', '#FF9900', '#E67300', '#DC3912']

# Create custom tickformatter based on savascript to pass to Bokeh
# hvplot.bar has no datetime axis support
formatter = FuncTickFormatter(code="""
    function getMonth(doy) {
        const date = new Date(2022, 0, doy);
        const month = date.toLocaleString('en-us', { month: 'short' })
        return month;
    }
    return getMonth(tick)
""")

# Specify ticks as doy
doy_month = [1,32,60,91,121,152,182,213,244,274,305,335]

barplot = count.hvplot.bar(groupby="time.year",
                           stacked=True, 
                           legend='top_left',
                           widget_type='individual',
                           widget_location="bottom",
                           cmap=colorlist,
                           line_alpha=0,
                           xticks=doy_month,
                           xformatter=formatter,
                           width=900, height=400,
                           ylabel='Lake coverage (%)',
                           xlabel='Date',
                           ylim=(0,100),
                           xlim=(0,365),
                           bar_width=1)

# Show hvplot.bar customization options
#hvplot.help('bar', generic=True, style=True)

barplot # show plot

#### Export yearly coverage barplot
We can create individual yearly barplots and export them as .png files.

In [ ]:
from bokeh.models import FuncTickFormatter
# Export specific years as .png
years = [2006, 2008, 2018] 
#years = list(range(1995, 2020)

# Colorlist with nice colors
colorlist = ['#66AA00','#AAAA11', '#FF9900', '#E67300', '#DC3912']

# Create custom tickformatter based on savascript to pass to Bokeh
# hvplot.bar has no datetime axis support
formatter = FuncTickFormatter(code="""
    function getMonth(doy) {
        const date = new Date(2022, 0, doy);
        const month = date.toLocaleString('en-us', { month: 'short' })
        return month;
    }
    return getMonth(tick)
""")

# Specify ticks as doy
doy_month = [1,32,60,91,121,152,182,213,244,274,305,335]

for year in years:
    count_slice = count.sel(time=slice(f'{year}-01-01', f'{year}-12-31'))
    year_barplot =  count_slice.hvplot.bar(
        title='ESA CCI Lakes v1.1\nLSWT coverage by quality flag',
        stacked=True, 
        legend='top_left',
        cmap=colorlist,
        line_alpha=0,
        xticks=doy_month,
        xformatter=formatter,
        width=900, height=500,
        ylabel='Lake coverage (%)',
        xlabel='Date',
        ylim=(0,100),
        xlim=(0,365),
        fontscale=1.5)

    # Export the plot of the current year using our export function
    fn = f'{lakename}-LSWT-QCoverage-{year}'
    exportPlot(year_barplot, fn)

### 2.4 Linear trend analysis
We can compute the linear trend for each lake cell using the [xarray.DataArray.polyfit](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.polyfit.html) function. Please note, that this is a very basic approach for trend analysis that does not take the seasonality and the inconsistent nature of the multi-source signal (e.g. data-gaps) into account.

In [ ]:
# Slice data with timeslice to get full years
timeslice = slice('1997-01-01', '2019-01-01')
lswt_slice = lswt.sel(time=timeslice)

polyfit_results = lswt_slice.polyfit(dim='time', deg=1, skipna=True).compute()

Plot the linear coefficients on a map.

In [ ]:
import math

ns_per_year = 3.154e+16
ns_per_decade = ns_per_year*10
trend_coef = polyfit_results.polyfit_coefficients.sel(degree=1)
trend_coef_perDec = trend_coef*ns_per_decade # convert °C/ns to °C/10y

vmin = np.nanpercentile(trend_coef_perDec, 5)
vmax = np.nanpercentile(trend_coef_perDec, 95)
vmin_tenth = math.ceil(vmin*10)/10
vmax_tenth = math.ceil(vmax*10)/10
clim_abs = max(abs(vmin_tenth), abs(vmax_tenth))

plot = trend_coef_perDec.hvplot(
    clabel='trend (°C/10y)', 
    label=f'Lake {lakename}, Linear trend',
    geo=True, 
    tiles='StamenTerrainRetina', # plot backgroundmap
    #features={'rivers':'10m'},
    cmap='bwr',
    clim=(-clim_abs, clim_abs),
    crs=crs
)

plot

In [ ]:
# Export plot
fn = f'{lakename}-LSWT-LinTrend-{timeslice.start}-{timeslice.stop}'
exportPlot(plot, fn)

Calculate the mean trend (°C/10y) over all lake cells.

In [ ]:
trend_coef_perDec.mean().values